In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import timeit

import os
import sys
root_path = os.getcwd() #+ "/scripts/personal/rick"
sys.path.append(str(root_path))

from quaos.core.prime_Functions_Andrew import *
from quaos.core.prime_Functions_quditV2 import *
np.set_printoptions(linewidth=200)

In [ ]:
def xi(a, d):
    """
    Computes the a-th eigenvalue of a pauli with dimension d.
    
    Parameters:
        a (int): The integer to compute the eigenvalue for.
        d (int): The dimension of the pauli to use.
    
    Returns:
        complex: The computed eigenvalue.
    """
    return np.exp(2 * np.pi * 1j * a / d)

def random_pauli_hamiltonian(num_paulis, qudit_dims,mode='rand'):
    """
    Generates a random Pauli Hamiltonian with the given number of Pauli operators.
    
    Parameters:
        num_paulis (int): Number of Pauli operators to generate.
        qudit_dims (list): List of dimensions for each qudit.
    
    Returns:
        tuple: A set of random Pauli operators and corresponding coefficients.
    """
    q2 = np.repeat(qudit_dims, 2)
    available_paulis = list(np.arange(int(np.prod(q2))))
    
    pauli_strings = []
    coefficients = []

    for _ in range(num_paulis):
        pauli_index = random.choice(available_paulis)
        available_paulis.remove(pauli_index)

        exponents = int_to_bases(pauli_index, q2)
        exponents_H = np.zeros_like(exponents)
        phase_factor = 1
        pauli_str = ' '
        pauli_str_H = ' '

        for j in range(len(qudit_dims)):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_str += f"x{r}z{s} "
            exponents_H[2*j] = (-r) % qudit_dims[j]
            exponents_H[2*j+1] = (-s) % qudit_dims[j]
            pauli_str_H += f"x{exponents_H[2*j]}z{exponents_H[2*j+1]} "
            
            omega = np.exp(2 * np.pi * 1j / qudit_dims[j])
            phase_factor *= omega**(r * s)
        
        pauli_strings.append(pauli_str.strip())
        if mode == 'rand':
            coeff = np.random.normal(0, 1) + 1j * np.random.normal(0, 1)
        elif mode == 'one':
            coeff = 1 + 0*1j
        
        if not np.array_equal(exponents, exponents_H):
            conjugate_index = bases_to_int(exponents_H, q2)
            coefficients.append(coeff)
            coefficients.append(np.conj(coeff) * phase_factor)
            available_paulis.remove(conjugate_index)
            pauli_strings.append(pauli_str_H.strip())
        else:
            coefficients.append(coeff.real)

    return string_to_pauli(pauli_strings, dims=qudit_dims, phases=0), coefficients

def reduce_qudits(P):
    C,pivots = symplectic_RREF_qudit(P)
    return C,pivots

def symplectic_RREF_qudit(P):
    q = P.qudits()
    P1 = P.copy()
    C = circuit(q)
    pivots = []

    for i in range(q):
        C,pivots = symplectic_RREF_iter_qudit_(P1.copy(),C,pivots,i)
    P1 = act(P1,C)

    removable_qubits = set(range(q))-set([pivot[1] for pivot in pivots])
    conditional_qubits = sorted(set(range(q))-removable_qubits-set([pivot[1] for pivot in pivots if pivot[2]=='Z']))
    if any(conditional_qubits):
        g = gate(H,conditional_qubits)
        C.add_gates_(g)
        P1 = act(P1,g)

    return C,sorted(pivots,key=lambda x:x[1])

def number_of_SUM_X(r_control,r_target,d):
    N = 1
    while (r_target + N*r_control)%d != 0:
        if N > d:
            raise Exception('Error in Exponents r_control = '+str(r_control)+' r_target = '+ str(r_target))
        N += 1
        
    return(N)

def number_of_SUM_Z(s_control,s_target,d):
    N = 1
    while (s_control - N*s_target)%d != 0:
        if N > d:
            raise Exception('Error in Exponents s_control = '+str(s_control)+' s_target = '+ str(s_target))
        N += 1
        
    return(N)

def number_of_S(x_exp,z_exp,d):
    N = 1
    while (x_exp*N + z_exp)%d != 0:
        if N > d:
            raise Exception('Error in Exponents x_exp = '+str(x_exp)+' z_exp = '+ str(z_exp))
        N += 1
        
        
    return(N)

def cancel_X(P1,a,a1,C,q_max):
    gg = []
    for i in range(a+1,q_max):
        if P1.X[a1,i]:
            gg += [gate(CX,[a,i])]*number_of_SUM_X(P1.X[a1,a],P1.X[a1,i],P1.dims[i])
    C.add_gates_(gg)     
    P1 = act(P1,gg)
    return(P1,C)

def cancel_Z(P1,a,a1,C,q_max):
    gg = []
    gg += [gate(H,[a])]
    for i in range(a+1,q_max):
        if P1.Z[a1,i]:
            gg += [gate(CX,[i,a])]*number_of_SUM_Z(P1.Z[a1,i],P1.X[a1,a],P1.dims[i])
    gg += [gate(H,[a])]
    C.add_gates_(gg)
    P1 = act(P1,gg)
    return(P1,C)

def cancel_Y(P1,a,a1,C):
    g = [gate(S,[a])]*number_of_S(P1.X[a1,a],P1.Z[a1,a],P1.dims[a])
    C.add_gates_(g)
    P1 = act(P1,g)
    return(P1,C)

def symplectic_RREF_iter_qudit_(P,C,pivots,a):
    p,q = P.paulis(),P.qudits()
    P = act(P,C)
    q_max = q
    for i in range(len(P.dims)-a):
        if P.dims[a+i] != P.dims[a]:
            q_max = a + i - 1
            break

    if any(P.X[:,a]) or any(P.Z[:,a]):
        if not any(P.X[:,a]):
            g = gate(H,[a])
            C.add_gates_(g)
            P = act(P,g)

        a1 = min(i for i in range(p) if P.X[i,a])
        pivots.append((a1,a,'X'))

        if any(P.X[a1,i] for i in range(a+1,q_max)):
            P,C = cancel_X(P,a,a1,C,q_max)

        if any(P.Z[a1,i] for i in range(a+1,q_max)):
            P,C = cancel_Z(P,a,a1,C,q_max)

        # if Pauli a1, qubit a is Y, add S gate to make it X
        if P.Z[a1,a]:
            P,C = cancel_Y(P,a,a1,C)


    if any(P.Z[:,a]):
        a1 = min(i for i in range(p) if P.Z[i,a])
        pivots.append((a1,a,'Z'))

        g = gate(H,[a])
        C.add_gates_(g)
        P = act(P,g)


        # add CX gates to cancel out all non-zero X-parts on Pauli a1, qubits > a
        if any(P.X[a1,i] for i in range(a+1,q_max)):
            P1,C = cancel_X(P,a,a1,C,q_max)

        # add CZ gates to cancel out all non-zero Z-parts on Pauli a1, qubits > a
        if any(P.Z[a1,i] for i in range(a+1,q_max)):
            P,C = cancel_Z(P,a,a1,C,q_max)

        # if Pauli a1, qubit a is Y, add S gate to make it X
        if P.Z[a1,a]:
            P,C = cancel_Y(P,a,a1,C)

        g = gate(H,[a])
        C.add_gates_(g)
        P = act(P,g)
    
    return C,pivots

def reduced_qudit_Hamiltonian(P,cc):
    PP1, ccc1 = [], []
    p,q = P.paulis(),P.qudits()
    C,pivots = reduce_qudits(P)
    P1 = act(P,C)
    P1.print()

    removable_qudits = set(range(q))-set([pivot[1] for pivot in pivots])
    conditional_qudits = set(range(q))-removable_qudits-set([pivot[1] for pivot in pivots if pivot[2]=='Z'])
    removable_qudits = sorted(removable_qudits)
    conditional_qudits = sorted(conditional_qudits)
    
    conditional_factors = []
    for i0 in range(p):
        conditional_factors.append([P1.Z[i0,i1] for i1 in conditional_qudits])

    P2 = P1.copy()
    P2.delete_qudits_(set(removable_qudits)|set(conditional_qudits))
    print("Q",q)
    print("R",len(removable_qudits))
    print("C",len(conditional_qudits))
    print("Removed",removable_qudits)
    print("Conditioned",conditional_qudits)
    if len(conditional_qudits) > 0:
        Pz_ss = []
        for i in conditional_qudits:
            for k in range(1,P.dims[i]):
                ss = ""
                for j in range(q):
                    ss += "x0"
                    if i == j:
                        ss += "z"+str(k)
                    else:
                        ss += "z0"
                    ss += " "
                ss = ss[:-1]
                Pz_ss.append(ss)

        Pz = string_to_pauli(Pz_ss,dims=P.dims)
        Pz = act_inverse(Pz,C)
        print("Pauli Symmetries:")
        Pz.print()

    for conditions in itertools.product(*[np.arange(P1.dims[conditional_qudits[i]]) for i in range(len(conditional_qudits))],repeat=1):
        cc1 = [cc[i0] * xi(np.sum(np.array(conditions) * np.array(conditional_factors[i0]) * P2.lcm/np.array(P1.dims[conditional_qudits])),P2.lcm) for i0 in range(p)]
        #cc1 = [cc[i0] * xi(np.sum(np.array(conditions) * np.array(conditional_factors[i0]))%P2.lcm, P2.lcm) for i0 in range(p)]
        cc2 = np.copy(cc1)
        P3 = P2.copy()
        P2_ss = pauli_to_string(P2)[0]
        
        # find duplicates
        duplicates = []
        for i in range(P2.paulis()):
            for j in range(i+1,P2.paulis()):
                if P2_ss[i] == P2_ss[j]:
                    already_a_duplicate = False
                    for k,dd in enumerate(duplicates):
                        if i in dd or j in dd:
                            duplicates[k] = dd | {i,j}
                            already_a_duplicate = True
                    if already_a_duplicate == False:
                        duplicates.append({i,j})
        
        # unify duplicates
        excess_paulis = []
        for dd in duplicates:
            i = sorted(list(dd))[0]
            cc2[i] = np.sum([cc1[j]*xi(P2.phases[j],P2.lcm) for j in dd])
            P3.phases[i] = 0
            excess_paulis += [j for j in sorted(list(dd))[1:]]
        P3.delete_paulis_(excess_paulis)
        cc3 = np.delete(cc2,excess_paulis)
        
        # add a part to remove paulis with coeficients that are 0
        zero_paulis = []
        for i,c in enumerate(cc3):
            if np.abs(c) < 10**(-10):
                zero_paulis.append(i)
        P3.delete_paulis_(zero_paulis)
        cc3 = np.delete(cc3,zero_paulis)
        PP1.append(P3)
        ccc1.append(cc3)
    
    return PP1,ccc1,C
    

In [3]:
P,cc = random_pauli_hamiltonian(6, [3,3,3,3,3,3,3],mode='one')
P.print()
P.print_symplectic()

psi = ground_state(P,cc)
print("GSE:",Hamiltonian_Mean(P,cc,psi))

x0z2 x1z0 x2z2 x1z0 x1z2 x0z2 x1z1
x0z1 x2z0 x1z1 x2z0 x2z1 x0z1 x2z2
x0z0 x2z1 x0z0 x2z1 x2z1 x0z0 x2z0
x0z0 x1z2 x0z0 x1z2 x1z2 x0z0 x1z0
x1z2 x1z1 x1z2 x1z2 x2z1 x2z0 x2z0
x2z1 x2z2 x2z1 x2z1 x1z2 x1z0 x1z0
x1z2 x2z1 x2z2 x1z1 x1z0 x2z0 x1z0
x2z1 x1z2 x1z1 x2z2 x2z0 x1z0 x2z0
x2z2 x1z1 x2z2 x2z1 x2z1 x2z2 x1z0
x1z1 x2z2 x1z1 x1z2 x1z2 x1z1 x2z0
x2z0 x1z2 x1z0 x1z1 x1z0 x1z0 x2z2
x1z0 x2z1 x2z0 x2z2 x2z0 x2z0 x1z1
3333333 3333333
------- -------
0121101 2020221 0/3
0212202 1010112 0/3
0202202 0101100 0/3
0101101 0202200 0/3
1111222 2122100 0/3
2222111 1211200 0/3
1221121 2121000 0/3
2112212 1212000 0/3
2122221 2121120 0/3
1211112 1212210 0/3
2111112 0201002 0/3
1222221 0102001 0/3
GSE: (-5.213175728940628+1.2246399702065787e-14j)


In [4]:
C,pivots = reduce_qudits(P)
P1 = act(P,C)
P1.print()
print()
psi1 = ground_state(P1,cc)
print("GSE:",Hamiltonian_Mean(P1,cc,psi1))

x0z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z0 x2z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z2 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x2z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z2 x0z2 x0z0 x0z0 x0z0 x0z0 x0z0
x0z1 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0
x2z0 x1z2 x2z0 x0z0 x0z0 x0z0 x0z0
x1z0 x2z1 x1z0 x0z0 x0z0 x0z0 x0z0
x2z0 x0z2 x0z1 x0z0 x0z0 x0z0 x0z0
x1z0 x0z1 x0z2 x0z0 x0z0 x0z0 x0z0

GSE: (-5.213175728940629+6.877430543917569e-15j)


In [5]:
PP1,ccc1,C = reduced_qudit_Hamiltonian(P,cc)
print()
print('Reduced Hamiltonians')
for i,P1 in enumerate(PP1):
    print('Hamiltonian Nr.',i)
    P1.print()
    print()
    psi1 = ground_state(P1,ccc1[i])
    print("GSE:",Hamiltonian_Mean(P1,ccc1[i],psi1))
    print()

x0z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z0 x2z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z2 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x2z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z2 x0z2 x0z0 x0z0 x0z0 x0z0 x0z0
x0z1 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0
x2z0 x1z2 x2z0 x0z0 x0z0 x0z0 x0z0
x1z0 x2z1 x1z0 x0z0 x0z0 x0z0 x0z0
x2z0 x0z2 x0z1 x0z0 x0z0 x0z0 x0z0
x1z0 x0z1 x0z2 x0z0 x0z0 x0z0 x0z0


NameError: name 'dims' is not defined